###  Featuretools：理论与实例 ###
Featuretools是执行自动特征工程的框架。 它可以将事务和关系数据集转换为机器学习的特征矩阵。 特征对于机器学习问题是至关重要的，而目前的特征处理方法通常离不开人工的干涉，利用人类的直觉进行特征工程虽然可以按照我们的经验自由地筛选特征、组合特征，但凭直觉、经验的工程一定是有局限的，而且极其耗费时间。下图展示了典型的数据分析过程：包括分析人员提出特定的问题、数据工程师提取出可解释的变量，再由机器学习工程师对数据进行建模，如此迭代。 为了避免人工处理时的局限性和耗费，我们需要有一种能够自动完成特征工程的工具。
![](typical.png)
####  Deep Feature Synthesis ####
Deep Feature Synthesis（DFS）是一种处理关系型和事务性数据的自动化特征工程方法。本质上，这个算法遵循数据与基本字段的关系，通过一系列的数学函数运算，创造出最终的特征。由于顺序的堆叠计算，每个新的特征都具有一定的深度d,该算法由此得名。  
我们从一个具体的实例来了解这个算法提出的动机和原型问题。假设有一个关于电子商务的网站，我们希望能够得到能够描述用户行为的特征，例如某用户上次购买行为发生在多久之前，该用户的购物频率等等。
![](scheme.png)  
上图表示了电商网站的一种数据库关系方案，其中有4个实体，从A到B的一个箭头代表A引用B（外键）。我们输入给DFS算法的数据是相互联系的实体集和与之相关的表，每个实体集中实例的特征不外乎数值型、类别型、时间戳和文本。  
为了方便描述，我们把给定数据集中的实体集记为$E^{1……k}$,每个实体表有$1……J$个特征，其中的某项${x^k}_{i,j}$为第k个实体集中第i个实例的第j个特征。  
在给定实体、对应的数据表和实体之间的关系下，接下来需要从两个不同的级别定义一些数学函数：实体级别和关系级别。
##### Entity Features（efeat） #####
通过特定的函数，将实体E中所有实例的第J个特征进行转换，可以得到实体特征。例如将类别型特征转换成特定的数字表示，将日期转换成数字等等。我们用一个简单的式子为这个过程做一个统一的表示：${x_{i,j}}^{'}=efeat(x:_{j},i)$ 。其中，efeat代表了我们进行的计算函数，例如我们使用累计概率分布作为我们的计算函数，那么得到的新特征就是原特征值j的累计概率（或者分位点）。  
##### Direct Features（dfeat）  和 Relational features (rfeat)#####
此外，另一个特征集通过联合分析两个关联的实体得到，两个实体集$E^l,E^k$有两种方式相互关联：前向和后向的。 前向关系存在于一个实体$E^l$中的实例m和实体$E^k$中的单个实例i之间，当实例i显示地依赖于m的时候，我们就称两个实体间有前向关系。比如上述电商的例子中Orders和Customers具有前向关系。 后向关系存在于实体$E^k$中的实例i和$E^l$中所有对实体$E^k$有前向关系的实例$m={1...M}$之间。 在上述的例子中，Customers和Orders之间就具有后向关系，因为某个用户可以下达多个订单。
针对前向关系，我们可以构造出direct features(dfeat), 这是$E^k$中实例i的特征可以直接迁移到$E^l$中的实例m。   
针对后向关系，可以构造出relational features(rfeat), 根据我们上述对后向关系的定义，这类特征是为实体$E^k$中的实例i构造的。先给出转换关系的统一公式：${x^k}_{i,j^{'}}=rfeat(x^l:_{j|e^k=i})$ $x^l:_{j|e^k=i}$代表的是实体$E^k$中的实例i所关联的在$E^l$中的实例的第j个特征的值的集合。用一个例子来简化对这个绕口概念的理解：假设$E^k$是购物者，其中某个用户实例为i,他所下达的订单实例位于订单实体集$E^l$中，j代表价格，是订单实例的一个特征，那么我们可以得到用户i所有订单的价格集合，rfeat代表我们选择的数学函数，可以有取最大值，取最小值，平均值等。
##### Algorithm #####
假设我们有包含实体集$E^{1……k}$的数据集，我们需要用DFS算法为其中一个实体$E^k$构造rfeat,dfeat,efeat,另外我们假设我们知道所有和$E^k$有前向关系和后向关系的实体，分别记为$E_F$和$E_B$。因为efeat是对存在的特征进行之间转换，只涉及单个实体，所以放在最后完成。而为了得到$E^k$中的rfeat，我们要用到$E_B$中的特征，所以，我们要先为$E_B$创造好所有的特征。以类似的方式为$E_k$加上dfeat前，我们也要首先完成对$E_F$的特征派生工作。这个过程可以用算法一表示：
![](algorithm1.jpg)  
算法一实际上是一个递归算法，$E_V$是指已经访问过的实体，用于避免循环添加特征。
现在我们通过一个实例来描述我们是如何为一个实体添加特征的。
![](example.png) 
针对Custermer这个实体，从上图中我们可以知道是如何为它添加一个新的特征“所有订单的平均价格”的。从“Product”实体开始，"Product"实体和“ProductOrder”实体是前向关系，所以我们可以为“ProductOrder”添加一个dfeat,即商品的价格。而"Oders"和“ProductOder”是后向关系，所以可以为"Oders"添加rfeat，一个订单中的商品总价格，而“Customers”和"Oders"是后向关系，所以又可以为"Customers"添加一个rfeat,即“所有订单的平均价格”，到这里可以看出该特征的深度为d=3。

##### 特征数量增长 #####
现在还有一个值得关注的问题，如何估量特征数量的增长，我们假设所有的实体有O（j）个特征，O（n）个前向关系和O（m）个后向关系。现在来进行特征数量z的计算，$z_i$表示我们进行i次迭代后时特征的数量。  
首先我们为O（m）个后向关系实体计算rfeat,如果我们有O(r)个refeat函数，那么我们可以为这m个实体都合成$O(r \cdot{z_{i-1}} )$个特征，那么此时总共就新增了$O(r \cdot{z_{i-1} \cdot{m}} )$个特征。 下一步，为前向关系添加$O( {z_{i-1}}\cdot{n}  )$个特征，那么我们总共可以为我们的目标实体添加$O( {z_{i-1}}\cdot{(rm+n)}  )$个新特征。 在此基础上，假设我们有O（e）个efeat函数，可以新增$O(e\cdot{j}+ e{z_{i-1}}\cdot{(rm+n)}  )$个efeat特征。 
结合rfeat,dfeat,efeat,$z_i=O(z_{i-1}\cdot{(r \cdot {m} +n)(e+1 +e\cdot{j})}$ 。
最终关于z的封闭式为$$z_i=O((ej) \sum_{u=0}^i{(rm+n)^u(e+1)^u)}$$

#### 机器学习预测通道 ####
为了使用DFS创造的特征，作者实现了一个机器学习的算法通道。第一步是构造一个预测问题，选取一个特征集中的一个特征作为预测的目标值，之后集成那些方便使用的特征进行预测，如果某个特征依赖于目标值（在作为目标值的特征生成之后才能被生成的特征），那么这个特征不能作为模型使用的训练特征。
##### 数据预处理#####
将Null值数据移除，再将类别型变量用One-hot编码转换，再经过变量的归一化就完成了数据预处理工作。
##### 特征选择和降维#####
DFS为每个实体生成了大量的特征，为了完成特征选择和降维，在机器学习通道中采用了两个方法：截断奇异值分界转换后选取$n_c$个SVD中的成分；然后计算每个SVD特征关于目标值的f值，排序后从中选取最高的$ \gamma \%$个特征。
##### 模型 #####
用K-means算法将数据集进行聚类，类数为k,作为数据的标签，然后采用随机森林算法，每棵树的深度为$m_d$,选取的特征比例为$\beta$。 因为在分类的问题中，有些目标值代表性不足，所以可以为这些代表性不足的类重新赋予权重rr。
现在总结一下我们现有的超参数，SVD成分n，决策树的深度$m_d$,特征比例$\beta$以及类权重rr。
##### 超参数优化 #####
作者采用高斯连接过程（Gaussian Copula Process，GCP）来进行超参数的调优，GCP用来模拟参数选择和整个机器学习路径的性能关系f。然后我们对参数进行新的采样再根据这个参数选择进行预测，衡量路径的性能。最后我们采取选择策略来决定所要选择的参数。
模拟：典型的高斯过程可以用来模拟数据集中有限个数据点${\overline{p}}_{1,……，N}$的f关系。$f(p_i)$在$R^n$中具有多元高斯分布，在本文中，作者在连接过程的基础上提出一种新的参数优化方法，它通过$R^n \rightarrow R^n$的$ \Psi $变换来进行映射，将$f(p_i)$转换为$ \Psi 。 f(p_i) $,然后再按照高斯过程建模。



#### 实例 ####

In [1]:
import featuretools as ft
data = ft.demo.load_mock_customer()

In [2]:
data

{'customers':    customer_id zip_code  join_date
 0            1    60091 2008-01-01
 1            2    02139 2008-02-20
 2            3    02139 2008-04-10
 3            4    60091 2008-05-30
 4            5    02139 2008-07-19, 'products':    product_id brand
 0           1     B
 1           2     B
 2           3     C
 3           4     A
 4           5     C, 'sessions':     session_id  customer_id   device       session_start
 0            1            1  desktop 2014-01-01 00:00:00
 1            2            1  desktop 2014-01-01 00:17:20
 2            3            5   mobile 2014-01-01 00:28:10
 3            4            3   mobile 2014-01-01 00:43:20
 4            5            2   tablet 2014-01-01 01:10:25
 5            6            1  desktop 2014-01-01 01:22:20
 6            7            2  desktop 2014-01-01 01:40:45
 7            8            2   mobile 2014-01-01 01:55:55
 8            9            1  desktop 2014-01-01 02:15:25
 9           10            2   mobile 201

上面是实例中的模拟数据，它包含了三个实体，customers: 其中每个实例是一个独一的用户，他有会话期。sessions:每个会话期有对应的属性，transactions: 发生会话期的事件列表。

In [3]:
customers_df = data["customers"]
print (customers_df)
sessions_df = data["sessions"]
print (sessions_df.sample(5))
transactions_df = data["transactions"]

print (transactions_df.sample(5))

   customer_id zip_code  join_date
0            1    60091 2008-01-01
1            2    02139 2008-02-20
2            3    02139 2008-04-10
3            4    60091 2008-05-30
4            5    02139 2008-07-19
    session_id  customer_id   device       session_start
16          17            4   mobile 2014-01-01 04:02:40
34          35            1  desktop 2014-01-01 08:45:25
4            5            2   tablet 2014-01-01 01:10:25
0            1            1  desktop 2014-01-01 00:00:00
29          30            4  desktop 2014-01-01 07:29:35
     transaction_id  session_id    transaction_time product_id  amount
481             441          34 2014-01-01 08:41:05          1   81.15
418              84          30 2014-01-01 07:32:50          4  149.02
19               85           2 2014-01-01 00:20:35          4  148.14
132             377           9 2014-01-01 02:23:00          4  112.07
148             109          10 2014-01-01 02:40:20          4   18.40


首先我们定义一个包含所有实体的字典：

In [4]:
entities = {
"customers" : (customers_df, "customer_id"),
"sessions" : (sessions_df, "session_id", "session_start"),
"transactions" : (transactions_df, "transaction_id", "transaction_time")
}

其次，我们指定这些实体是如何相关的。 当两个实体具有一对多关系时，我们把“一”对应的那个实体称之为parent。 父母和孩子之间的关系是这样定义的：
(parent_entity, parent_variable, child_entity, child_variable),在我们的实例中就有这样的关系：

In [5]:
relationships = [("sessions", "session_id", "transactions", "session_id"),
                 ("customers", "customer_id", "sessions", "customer_id")]

下面我们用dfs算法为customer实体创造新的对象：

In [6]:
feature_matrix_customers, features_defs = ft.dfs(entities=entities,
                                                 relationships=relationships,
                                                 target_entity="customers")

In [7]:
feature_matrix_customers

,zip_code,COUNT(sessions),NUM_UNIQUE(sessions.device),MODE(sessions.device),SUM(transactions.amount),STD(transactions.amount),MAX(transactions.amount),SKEW(transactions.amount),MIN(transactions.amount),MEAN(transactions.amount),...,NUM_UNIQUE(sessions.MODE(transactions.product_id)),NUM_UNIQUE(sessions.DAY(session_start)),NUM_UNIQUE(sessions.YEAR(session_start)),NUM_UNIQUE(sessions.MONTH(session_start)),NUM_UNIQUE(sessions.WEEKDAY(session_start)),MODE(sessions.MODE(transactions.product_id)),MODE(sessions.DAY(session_start)),MODE(sessions.YEAR(session_start)),MODE(sessions.MONTH(session_start)),MODE(sessions.WEEKDAY(session_start))
customer_id,,,,,,,,,,,,,,,,,,,,,
1,60091,10,3,desktop,10236.77,42.673267,149.95,0.070041,5.60,78.143282,...,3,1,1,1,1,1,1,2014,1,2
2,02139,8,3,mobile,9118.81,43.204771,149.15,0.028647,5.81,74.744344,...,5,1,1,1,1,1,1,2014,1,2
3,02139,5,2,desktop,5758.24,40.127924,147.73,0.070814,6.78,73.823590,...,4,1,1,1,1,3,1,2014,1,2
4,60091,8,3,desktop,8205.28,41.857208,149.56,0.087986,5.73,73.921441,...,5,1,1,1,1,1,1,2014,1,2
5,02139,4,3,tablet,4571.37,42.656189,148.17,0.085883,5.91,78.816724,...,3,1,1,1,1,2,1,2014,1,2


可以看到，相比于原始的字段，已经增加了很多新的特征。例如用户会话数。

以上就是featuretools自动进行特征工程的一个简单实例，此外我们来看一下featuretools的一些其他操作，首先我们先看一下另一个实体product:

In [8]:
products_df = data["products"]
products_df

,product_id,brand
0,1,B
1,2,B
2,3,C
3,4,A
4,5,C


In [9]:
es = ft.EntitySet(id="transactions")
es

Entityset: transactions
  Entities:
  Relationships:
    No relationships

#### 初始化一个实体集 #### 
我们初始化一个新的实体集es,id为transactions.

In [10]:
es = es.entity_from_dataframe(entity_id="transactions",
                                  dataframe=transactions_df,
                                  index="transaction_id",
                                  time_index="transaction_time",
                                  variable_types={"product_id": ft.variable_types.Categorical})

可以将transactions实体的datafram加载到新的实体集es中，这样所有原本在transactions实体中的列就成为了新实体集的属性。

In [11]:
es = es.entity_from_dataframe(entity_id="products",
                                  dataframe=products_df,
                                  index="product_id")
   

用同样的方法我们可以将product实体加入es中

In [12]:
new_relationship = ft.Relationship(es["products"]["product_id"],
                                     es["transactions"]["product_id"])
es = es.add_relationship(new_relationship)
es

Entityset: transactions
  Entities:
    transactions (shape = [500, 5])
    products (shape = [5, 2])
  Relationships:
    transactions.product_id -> products.product_id

之后我们可以为实体集中的两个实体添加关系，每种商品都可以在多次交易中出现，所以我们让商品作为父实体，交易作为子实体。

调用normalize_entity函数可以以原有的实体为基础创造出新的实体，并为二者添加关系。之后就可以用我们的实体集去完成特征创造的工作了：

In [13]:
feature_matrix, feature_defs = ft.dfs(entityset=es, target_entity="products") 


In [14]:
feature_matrix

,brand,SUM(transactions.session_id),SUM(transactions.amount),STD(transactions.session_id),STD(transactions.amount),MAX(transactions.session_id),MAX(transactions.amount),SKEW(transactions.session_id),SKEW(transactions.amount),MIN(transactions.session_id),...,MEAN(transactions.amount),COUNT(transactions),NUM_UNIQUE(transactions.DAY(transaction_time)),NUM_UNIQUE(transactions.YEAR(transaction_time)),NUM_UNIQUE(transactions.MONTH(transaction_time)),NUM_UNIQUE(transactions.WEEKDAY(transaction_time)),MODE(transactions.DAY(transaction_time)),MODE(transactions.YEAR(transaction_time)),MODE(transactions.MONTH(transaction_time)),MODE(transactions.WEEKDAY(transaction_time))
product_id,,,,,,,,,,,,,,,,,,,,,
1,B,1736,7046.84,10.067122,40.232770,35,148.86,0.056502,-0.027598,1,...,71.906531,98,1,1,1,1,1,2014,1,2
2,B,1677,7247.48,9.685635,39.083334,35,147.86,0.030685,0.180324,1,...,75.494583,96,1,1,1,1,1,2014,1,2
3,C,1796,7916.96,10.732463,41.647180,35,149.95,-0.102766,-0.075324,1,...,82.468333,96,1,1,1,1,1,2014,1,2
4,A,1967,8181.19,10.090767,44.354276,35,149.02,-0.006896,0.153199,1,...,75.056789,109,1,1,1,1,1,2014,1,2
5,C,1805,7498.00,10.813141,44.686334,35,149.56,-0.052114,0.087860,1,...,74.237624,101,1,1,1,1,1,2014,1,2


#### 元函数 ####
在进行DFS算法的时候，我们涉及到了一些用于创造新特征的元函数，例如Mean,Sum等等。例如我们还可以用TimeSincePrevious来计算用户两次session之间的间隔。  
元函数可以被分为Aggregation primitives和Transform primitives。  
Aggregation primitives: 这些原函数可以应用于具有父子关系的实体集中，它的输出是一个值，例如Count,Sum,AvgTimeBetween.
Transform primitives:这类元函数从一个实体中提取一个或者多个变量作为输入，产生一个新的变量给该实体，例如Hour,TimeSincePrevious,Absolute.


In [26]:
from featuretools.primitives import TimeSincePrevious, Mean
import featuretools as ft

es = ft.demo.load_mock_customer(return_entityset=True)
feature_defs = ft.dfs(entityset=es,
                    target_entity="customers",
                    agg_primitives=[Mean],
                    trans_primitives=[TimeSincePrevious])
feature_defs


(            zip_code  MEAN(transactions.amount)  \
 customer_id                                       
 1              60091                  78.143282   
 2              02139                  74.744344   
 3              02139                  73.823590   
 4              60091                  73.921441   
 5              02139                  78.816724   
 
              MEAN(sessions.MEAN(transactions.amount))  \
 customer_id                                             
 1                                           79.197651   
 2                                           74.530438   
 3                                           73.954024   
 4                                           73.084141   
 5                                           78.362236   
 
              MEAN(sessions.time_since_previous_by_customer_id)  
 customer_id                                                     
 1                                                  3502.777778  
 2                          

我们还可以使用自己定义的Aggregation primitives,例如下面定义了一个Absolute的元函数，我们可以仿照上面使用Mean元函数的例子使用它来添加特征。

In [27]:
from featuretools.primitives import make_agg_primitive, make_trans_primitive

from featuretools.variable_types import Text, Numeric

def absolute(column):
    return abs(column)


Absolute = make_trans_primitive(function=absolute,
                                input_types=[Numeric],
                                return_type=Numeric)

#### 时间处理 ####
当我们进行特征工程去训练模型来预测特征的时候，需要预测的特征值可能会与时间相关，在这种情况下，在截断时间之前计算特征值是极为重要的。如果不能妥当处理这个问题，就可能造成标签泄漏的情况，那么预测得到的新特征将毫无作用。下面这个例子展示 了“截断时间”的使用方法，它仅仅采用截断时间之前的实例来进行新特征的生成。
除了加上截断时间作为参数，我们还可以设置时间窗口， training_window="1 hour"代表我们仅仅使用截断时间前1小时内的事件进行处理。


In [31]:
import pandas as pd

cutoff_times = pd.DataFrame({"customer_id": [1, 2, 3, 4, 5],
                             "time": pd.date_range('2014-01-01 01:41:50', periods=5, freq='25min')})

In [32]:
feature_matrix, features = ft.dfs(entityset=es,
                                  target_entity="customers",
                                  cutoff_time=cutoff_times)

feature_matrix

,zip_code,COUNT(sessions),NUM_UNIQUE(sessions.device),MODE(sessions.device),SUM(transactions.amount),STD(transactions.amount),MAX(transactions.amount),SKEW(transactions.amount),MIN(transactions.amount),MEAN(transactions.amount),...,NUM_UNIQUE(sessions.MODE(transactions.product_id)),NUM_UNIQUE(sessions.DAY(session_start)),NUM_UNIQUE(sessions.YEAR(session_start)),NUM_UNIQUE(sessions.MONTH(session_start)),NUM_UNIQUE(sessions.WEEKDAY(session_start)),MODE(sessions.MODE(transactions.product_id)),MODE(sessions.DAY(session_start)),MODE(sessions.YEAR(session_start)),MODE(sessions.MONTH(session_start)),MODE(sessions.WEEKDAY(session_start))
customer_id,,,,,,,,,,,,,,,,,,,,,
1,60091,3,1.0,desktop,3342.76,43.143511,148.14,-0.024647,5.60,77.738605,...,3.0,1.0,1.0,1.0,1.0,1.0,1.0,2014.0,1.0,2.0
2,02139,3,3.0,desktop,2558.77,41.140046,139.23,0.212373,6.29,71.076944,...,2.0,1.0,1.0,1.0,1.0,1.0,1.0,2014.0,1.0,2.0
3,02139,1,1.0,mobile,2054.32,45.708958,147.73,-0.215072,8.70,82.172800,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2014.0,1.0,2.0
4,60091,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,02139,2,2.0,mobile,2296.42,39.139809,141.66,0.167792,20.91,79.186897,...,2.0,1.0,1.0,1.0,1.0,2.0,1.0,2014.0,1.0,2.0


#### 调整 ####
在Featuretools的DFS算法中，一些参数可以进行调整来改变输出。例如使用种子特征：种子特征是手动定义的，问题特定的，用户提供给DFS的特征。 然后，Deep Feature Synthesis会自动在这些特征之上堆叠新特征。
通过使用种子特征，我们可以在特征工程自动化中包含针对领域特定的知识。例如下面这个例子用expensive_purchase作为种子特征，使用PercentTrue作为元函数，把一个布尔变量转换成了数值变量。

In [35]:
import featuretools as ft
es = ft.demo.load_mock_customer(return_entityset=True)
from featuretools.primitives import PercentTrue

expensive_purchase = ft.Feature(es["transactions"]["amount"]) > 125

feature_matrix, feature_defs = ft.dfs(entityset=es,
                                       target_entity="customers",
                                       agg_primitives=[PercentTrue],
                                       seed_features=[expensive_purchase])
 

feature_matrix[['PERCENT_TRUE(transactions.amount > 125)']]

,PERCENT_TRUE(transactions.amount > 125)
customer_id,
1,0.213740
2,0.172131
3,0.115385
4,0.144144
5,0.224138


此外我们还可以用where从句使得我们可以针对满足特定条件的数据进行特征的运算。比如我们想看终端分别为桌面系统、移动终端、平板设备的会话期间的一些消费行为，就可以采用这种方法。

In [38]:
from featuretools.primitives import Count, AvgTimeBetween
es["sessions"]["device"].interesting_values = ["desktop", "mobile", "tablet"]
feature_matrix, feature_defs = ft.dfs(entityset=es,
                                           target_entity="customers",
                                           agg_primitives=[Count, AvgTimeBetween],
                                           where_primitives=[Count, AvgTimeBetween],
                                           trans_primitives=[])
feature_matrix[["COUNT(sessions WHERE device = tablet)", "AVG_TIME_BETWEEN(sessions.session_start WHERE device = tablet)"]]

,COUNT(sessions WHERE device = tablet),AVG_TIME_BETWEEN(sessions.session_start WHERE device = tablet)
customer_id,,
1,1.0,NaN
2,3.0,9295.0
3,0.0,NaN
4,3.0,4160.0
5,2.0,15860.0
